In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
from feature_store import Client
from feature_store.feature import FeatureKind
import sqlalchemy as sa

In [3]:
client = Client()

In [4]:
client.get_available_features()

[<Feature neighbourhood>, <Feature room_type>, <Feature price>]

In [5]:
f = client.get_feature("price")

In [6]:
f

<Feature price>

In [7]:
data = client.get_dataset(["neighbourhood", "room_type"])

In [8]:
data

Dataset(features=[<Feature neighbourhood>, <Feature room_type>])

In [9]:
data = data + f

In [15]:
data.to_pandas().date.value_counts()

2023-03-31    14759
2022-12-29    13820
2022-09-24    13815
Name: date, dtype: int64